<a href="https://colab.research.google.com/github/JLBros10/AI-Science-Fair/blob/main/Weather_NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/flatplanet/Pytorch-Tutorial-Youtube/blob/main/simple_NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
class Model(nn.Module):
  def __init__(self, in_features=8, h1=20, h2=10, out_features=9):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1, h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x


In [ ]:
torch.manual_seed(41)
model = Model()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
my_df = pd.read_csv("/content/drive/MyDrive/SF/Datasets for Train n' Test/weather_prediction_dataset.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
my_df = my_df.iloc[:,2:11]
my_df.tail()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max
3649,7,0.82,1.0084,0.28,0.42,0.3,3.2,1.0,4.8
3650,7,0.92,1.0028,0.22,1.68,0.2,4.5,2.4,10.0
3651,8,0.92,0.9979,0.07,1.54,0.0,8.5,7.5,11.5
3652,7,0.93,0.9958,0.17,0.57,0.1,6.6,4.3,7.9
3653,8,0.93,0.9965,0.08,0.56,0.0,2.9,-0.2,4.8


In [ ]:
X = my_df.drop(['BASEL_cloud_cover'], axis=1)
y = my_df['BASEL_cloud_cover']

In [ ]:
X = X.values
y = y.values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

In [ ]:
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time

total_hours = 1
seconds_per_hour = 3600
ten_minutes = 60

losses = []

start_time = time.time()
last_logged_time = start_time
end_time = start_time + total_hours * seconds_per_hour

while time.time() < end_time:

    y_pred = model.forward(X_train)

    loss = criterion(y_pred, y_train)


    if time.time() - last_logged_time >= ten_minutes:
        elapsed_time = time.time() - start_time
        print(f'Elapsed Time: {elapsed_time / 3600:.2f} hours, Loss: {loss.detach().numpy()}')
        last_logged_time = time.time()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(f'Elapsed Time: {elapsed_time / 3600:.2f} hours, Loss: {loss.detach().numpy()}')

Elapsed Time: 0.02 hours, Loss: 1.1919187307357788
Elapsed Time: 0.03 hours, Loss: 1.181801199913025
Elapsed Time: 0.05 hours, Loss: 1.1763135194778442
Elapsed Time: 0.07 hours, Loss: 1.1723792552947998
Elapsed Time: 0.08 hours, Loss: 1.1688436269760132
Elapsed Time: 0.10 hours, Loss: 1.1604845523834229
Elapsed Time: 0.12 hours, Loss: 1.155880093574524
Elapsed Time: 0.13 hours, Loss: 1.1553586721420288
Elapsed Time: 0.15 hours, Loss: 1.1532492637634277
Elapsed Time: 0.17 hours, Loss: 1.1514390707015991
Elapsed Time: 0.18 hours, Loss: 1.150833249092102
Elapsed Time: 0.20 hours, Loss: 1.1540063619613647
Elapsed Time: 0.22 hours, Loss: 1.1526691913604736
Elapsed Time: 0.23 hours, Loss: 1.1479007005691528
Elapsed Time: 0.25 hours, Loss: 1.1459412574768066
Elapsed Time: 0.27 hours, Loss: 1.1449161767959595
Elapsed Time: 0.28 hours, Loss: 1.1441994905471802
Elapsed Time: 0.30 hours, Loss: 1.1435654163360596
Elapsed Time: 0.32 hours, Loss: 1.142716884613037
Elapsed Time: 0.33 hours, Loss: 1.1

In [ ]:
with torch.no_grad():
  y_eval = model.forward(X_test)
  loss = criterion(y_eval, y_test)


In [ ]:
loss

In [ ]:
import time
from functools import total_ordering
correct = 0
total_possible = 0
with torch.no_grad():
  for i, data in enumerate(X_test):
    total_possible += 1
    y_val = model.forward(data)

    #print(f'{i+1}. {str(y_val):38} \t {y_test[i]}')

    if y_val.argmax().item() == y_test[i]:
      correct += 1

print(f'We got {correct}/{total_possible} correct')